In [2]:
import sys
sys.path.append('../../flask-insight/')

from flask import render_template, request
from bill_app import app
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import pandas as pd
import psycopg2
from bill_app import models 

In [3]:
# Python code to connect to Postgres
# You may need to modify this based on your OS,
# as detailed in the postgres dev setup materials.
user = 'melissaferrari'  # add your Postgres username here
host = 'localhost'
dbname = 'congressional_bills'
db = create_engine('postgres://%s%s/%s' % (user, host, dbname))
con = None
con = psycopg2.connect(database=dbname,
                       user=user)

In [ ]:
@app.route('/')
@app.route('/index')
def index():
    return render_template("index.html",
                           title='Home',
                           user={'nickname': 'Melissa'},)

In [ ]:
@app.route('/db')
def bill_page():
    sql_query = """
                SELECT * FROM bills WHERE status=1;
                """
    
    query_results = pd.read_sql_query(sql_query, con)
    bills = ""
    print(query_results[:10])
    for i in range(0, 10):
        bills += query_results.iloc[i]['official_title']
        bills += "<br>"
    return bills

In [ ]:
@app.route('/db_fancy')
def bills_page_fancy():
    sql_query = """
                SELECT bill_id, official_title, subjects_top_term FROM bills WHERE status=1;
                """
    query_results = pd.read_sql_query(sql_query, con)
    bills = []
    for i in range(0, query_results.shape[0]):
        bills.append(dict(bill_id=query_results.iloc[i]['bill_id'],
                          official_title=query_results.iloc[i]['official_title'],
                          subjects_top_term=query_results.iloc[i]['subjects_top_term']))
    return render_template('bill_list.html', bills=bills)

In [ ]:
@app.route('/input')
def bills_input():
    query_list = "SELECT bill_id FROM bills;"
    query_list_results = pd.read_sql_query(query_list, con)

    return render_template("input.html",
                           title='Database selector',
                           bill_list=query_list_results)

In [ ]:
@app.route('/output')
def bills_output():
    # pull 'bill_id' from input field and store it
    # bill_id = request.args.get('bill_id')
    bill_id = 'hr4764-114'

    # just select the bill from the database
    query_info = "SELECT bill_id, official_title, subjects_top_term FROM bills WHERE bill_id='%s';" % bill_id
    #query_text = "SELECT bill_ix, text FROM bill_text WHERE bill_ix = (SELECT ls.id FROM bills ls WHERE bill_id='%s');" % bill_id
    query_text = "SELECT bill_ix, text FROM bill_text bt INNER JOIN bills b ON bt.bill_ix = b.id WHERE b.bill_id = '%s';" % bill_id
    
    query_info_results = pd.read_sql_query(query_info, con)
    query_text_results = pd.read_sql_query(query_text, con)
    
    bill_id = query_info_results.iloc[0]['bill_id'],
    official_title = query_info_results.iloc[0]['official_title'],
    subjects_top_term = query_info_results.iloc[0]['subjects_top_term']
    
    string_xml = query_text_results.iloc[0]['text']
    summarization_result = models.do_summarization(string_xml)

    return render_template("output.html",
                           summarization_result=summarization_result, 
                           bill_id=bill_id,
                           official_title=official_title,
                           subjects_top_term=subjects_top_term)

In [5]:
bill_id = 'hr4764-114'

In [6]:
query_info = "SELECT bill_id, official_title, subjects_top_term FROM bills WHERE bill_id='%s';" % bill_id

In [7]:
query_info

"SELECT bill_id, official_title, subjects_top_term FROM bills WHERE bill_id='hr4764-114';"

In [8]:
query_info_results = pd.read_sql_query(query_info, con)

In [12]:
query_info_results

,bill_id,official_title,subjects_top_term
0,hr4764-114,To direct the Secretary of Veterans Affairs to...,Armed forces and national security


In [10]:
query_info_results.iloc[0]['bill_id']

'hr4764-114'

In [13]:
query_list = "SELECT bill_id FROM bills;"
query_list_results = pd.read_sql_query(query_list, con)